<a href="https://colab.research.google.com/github/ANISHG-26/ELG6131-Dementia-EDA/blob/main/ELG_6131_Dementia_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>